In [11]:
# merf psd 0.7742113806103438
# merf after reref to avg mastoids 0.7436573207110183
#psd - pid - all - KNeighborsClassifier          - 0.750

#TODO: ADD OTHER REGRESSORS TO MERF. 
#TODO: ensure that we have balanced data per participant in training/testing. 
#TODO: see how hold-one-out participant training works; see if adding some training data per person works. 

# TODO ML: hold one out participant
# TODO ML: collapse with extra features -> sd/min/max/quantiles?
# TODO ML: validation set 

import pandas as pd
import os
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, \
                             AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture
from sklearn.gaussian_process import GaussianProcessClassifier
#from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import KFold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
from merf import MERF
from library import *

MERF_MAX_ITER = 20

#suppress undefined metric warnings
from sklearn.exceptions import UndefinedMetricWarning

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

np.random.seed(RANDOM_STATE)

#F_FMT = 'bp_notch_smooth_avgref.csv'

Y_STYLE = 'elo'

LOWER_QUANTILE = 0.25
UPPER_QUANTILE = 0.75

COLLAPSE = False

# tried regressors: AdaBoostRegressor() -- 55
MERF_REGRESSOR = RandomForestRegressor(random_state=RANDOM_STATE)

# GaussianMixture() -> bad for everyone 
MODELS = [ RandomForestClassifier(random_state=RANDOM_STATE), svm.SVC(kernel='rbf'),
           svm.SVC(kernel='linear'), svm.SVC(kernel='poly'), svm.SVC(kernel='sigmoid'),
            KNeighborsClassifier(n_neighbors=3), AdaBoostClassifier(), GradientBoostingClassifier(),
           GaussianNB(), GaussianProcessClassifier(), QuadraticDiscriminantAnalysis(), LinearDiscriminantAnalysis() ]

print(f"loading file: {DBFNAME.split('.csv')[0]}_psd_{FILTNAME}")
psd_df = pd.read_csv(f"{DBFNAME.split('.csv')[0]}_psd_{FILTNAME}", dtype = {'pid': str})
#raw_df = pd.read_csv(f"{DBFNAME.split('.csv')[0]}_raw_{FILTNAME}", dtype = {'pid': str})

# drop all rows that have 'nan' in pid
psd_df = psd_df[psd_df['pid'].notna()]
#raw_df = raw_df[raw_df['pid'].notna()]

PIDS = psd_df['pid'].unique()

# side effect - block_id is now globally unique. 
def extract_features(df, collapse_blocks=True):
    if collapse_blocks:
        df = df.groupby(['pid', 'block_id']).mean().reset_index()
      
    p_dfs = {}
    unique_id = 0
    for pid in df['pid'].unique():
        p_df = df[df['pid'] == pid].copy().reset_index(drop=True)
        
        for block_id in p_df['block_id'].unique():
            p_df.loc[p_df['block_id'] == block_id, 'block_id'] = unique_id
            unique_id += 1

        lower = p_df['elo'].quantile(LOWER_QUANTILE)
        upper = p_df['elo'].quantile(UPPER_QUANTILE)

        if Y_STYLE == 'elo':            
            p_df = p_df[(p_df['elo'] <= lower) | (p_df['elo'] >= upper)].reset_index(drop=True)
            p_df['y'] = p_df['elo'] > lower
        else:            
            p_df['y'] = p_df['solved']

        p_dfs[pid] = p_df
    
    return p_dfs
    
ALL_DFS = {
            #'raw': extract_features(raw_df), 
            'psd': extract_features(psd_df, collapse_blocks=COLLAPSE)
        }

for p in PIDS:
    for dtype in ALL_DFS:
        print(f"{p} - {dtype} - {len(ALL_DFS[dtype][p])}")

for dtype in ALL_DFS:
    all_X = []
    all_y = []
    all_P = []
    all_B = []
    for i, p in enumerate(PIDS):   
        print(p)     
        p_subset_df = ALL_DFS[dtype][p]    
        B = p_subset_df['block_id'].to_numpy()
        y = p_subset_df['y'].to_numpy()    
        X = p_subset_df.drop(columns=['y', 'pid', 'block_id', 'timestamp', 'elo', 'solved']).to_numpy()
        all_X.extend(X)
        all_y.extend(y)
        all_P.extend(pd.Series([i] * len(X)))
        all_B.extend(B)

        #runKFold(X, y, B, p, dtype)
    print('---------------------------------')
    print('---------------------------------')

    all_X = np.array(all_X)
    all_y = np.array(all_y)
    all_P = pd.Series(all_P)
    #runKFold(all_X, all_y, all_B, 'all', dtype)
    runKFold(all_X, all_y, all_B, 'all', dtype, merf=True, P=all_P)
    print('---------------------------------')
    print('---------------------------------')




loading file: study_p_5_5_psd_bp_notch_avgref.csv
2a7ba24a - psd - 6881
44dd5d31 - psd - 6254
92511e53 - psd - 8668
1bf79614 - psd - 5249
cf88d785 - psd - 12210
168dd1ac - psd - 12429
7cc59678 - psd - 8327
16230396 - psd - 7754
5c1034cf - psd - 9044
7848bec2 - psd - 5843
2a7ba24a
44dd5d31
92511e53
1bf79614
cf88d785
168dd1ac
7cc59678
16230396
5c1034cf
7848bec2
---------------------------------
---------------------------------


INFO     [merf.py:307] Training GLL is -160454.75383509905 at iteration 1.
INFO     [merf.py:307] Training GLL is -160665.8145211323 at iteration 2.
INFO     [merf.py:307] Training GLL is -160686.86373086614 at iteration 3.
INFO     [merf.py:307] Training GLL is -160798.39868076943 at iteration 4.
INFO     [merf.py:307] Training GLL is -160979.63221557578 at iteration 5.
INFO     [merf.py:307] Training GLL is -161048.37970965708 at iteration 6.
INFO     [merf.py:307] Training GLL is -161044.69469096072 at iteration 7.
INFO     [merf.py:307] Training GLL is -161091.51166000185 at iteration 8.
INFO     [merf.py:307] Training GLL is -160938.27543620756 at iteration 9.
INFO     [merf.py:307] Training GLL is -161038.355776976 at iteration 10.
INFO     [merf.py:307] Training GLL is -161257.3200944365 at iteration 11.
INFO     [merf.py:307] Training GLL is -161169.73650929728 at iteration 12.
INFO     [merf.py:307] Training GLL is -161232.60865318374 at iteration 13.
INFO     [merf.py:307] Tr

NameError: name 'mrf' is not defined

In [10]:
"""
runKFold 
inputs: 
    X: brain data
    y: labels
    B: block ids
    pid: participant id
"""

def runKFold(X, y, B, pid, datatype, merf=False, P=None):
    kf = KFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

    # unique block ids for the participant
    b_ids = np.unique(B)

    # this is janky but we can remove the other function
    if merf:
        MODELS = [MERF(MERF_REGRESSOR, max_iterations=MERF_MAX_ITER)]
        reports = { "MERF": [] }
    else:
        reports = { model: [] for model in MODELS }
   
    for i, (train_b_id_idxs, test_b_id_idxs) in enumerate(kf.split(b_ids)):        
        
        # print(train_b_ids)
        # print(test_b_ids)
        # print('-------------------')
        train_indices = [i for i in range(len(B)) if B[i] in b_ids[train_b_id_idxs]]
        test_indices = [i for i in range(len(B)) if B[i] in b_ids[test_b_id_idxs]]   
        
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]

        for model in MODELS:
            if merf:
                P_train, P_test = P[train_indices], P[test_indices]
                Z_train = np.ones((len(X_train), 1))
                Z_test = np.ones((len(X_test), 1))                
                model.fit(X_train, Z_train, P_train, y_train)
                
                preds = model.predict(X_test, Z_test, P_test)
                preds = [pred > 0.5 for pred in preds] # duh this isn't super great 
            else:
                model.fit(X_train, y_train)
                preds = model.predict(X_test)
        
            report = classification_report(y_test, preds, output_dict=True)
            reports[model].append(report['macro avg']['f1-score'])
    
    avgs = {model:np.mean(reports[model]) for model in reports}
    max_name_length = max(len(model.__class__.__name__) for model in avgs)

    # print sorted in desceding classification order
    avgs = {k: v for k, v in sorted(avgs.items(), key=lambda item: item[1], reverse=True)}
    for model, avg in avgs.items():
        print(f"{datatype} - pid - {pid} - {model.__class__.__name__:<{max_name_length}} - {avg:.3f}")
    
    print('---------------------------------')
    

In [ ]:
print(model)